In [117]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import pandas as pd
from nltk.stem.snowball import SnowballStemmer
import string
from string import digits
nltk.download('punkt')
nltk.download('stopwords')
import xgboost as xgb
import numpy as np
import sklearn as sk
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import ensemble, metrics, model_selection, naive_bayes, pipeline
from stop_words import get_stop_words
import gensim

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Radames\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Radames\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [118]:
df_train = pd.read_csv("../data/train.csv")
df_test = pd.read_csv("../data/test.csv")
# 3 столбца - id, text, author
df_train.head(n=3)

,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP


In [119]:
# нулей не должно быть!


def sum_up_word2vec_array(clnd_text):
    
    total = None
    
    for word in clnd_text:
        if word in w2v:
            if total is None:
                total = w2v[word]
            else:
                total = np.add(total, w2v[word])
                
    if total is None:
        return np.zeros(w2v_array_len)
    else:
        return total
    

In [120]:
# нулей не должно быть!


def avg_up_word2vec_array(clnd_text):
    
    total = None
    
    for word in clnd_text:
        if word in w2v:
            if total is None:
                total = w2v[word]
            else:                
                total = np.mean([total, w2v[word]], axis=0)
                
                
    if total is None:
        return np.zeros(w2v_array_len)
    else:
        return total
    

In [121]:
remove_digits = str.maketrans('', '', digits)
def tokenize_stem(file_text):
    #firstly let's apply nltk tokenization
    file_text = file_text.translate(remove_digits)
    
    tokens = nltk.word_tokenize(file_text)

    #let's delete punctuation symbols
    tokens = [i for i in tokens if ( i not in string.punctuation )]

    #deleting stop_words
    stop_words = stopwords.words('english')
    tokens = [i for i in tokens if ( i not in stop_words )]

    #cleaning words
    stemmer = SnowballStemmer("english")
    
    tokens = [stemmer.stem(i) for i in tokens]

    return tokens

In [122]:
df_train['cleaned_text'] = df_train.text.apply(tokenize_stem)
df_train['cleaned_text_string'] = df_train.cleaned_text.apply(' '.join)
df_train.head(n=3)
eng_stopwords = set(stopwords.words('english')).union(set(get_stop_words('english')))

In [123]:
def lexical_diversity(file_text):
    file_text = file_text.translate(remove_digits)
    try:
        tokens = nltk.word_tokenize(file_text)
    except:
        nltk.download('punkt')
        tokens = nltk.word_tokenize(file_text)
        

    #let's delete punctuation symbols
    tokens = [i for i in tokens if ( i not in string.punctuation )]

    #cleaning words
    stemmer = SnowballStemmer("english")
    
    tokens = [stemmer.stem(i) for i in tokens]

    return len(set(tokens))/len(tokens)

In [124]:
# вытаскиваем "значимые" слова




raw_documents_authors = ['', '', '']


for index, row in df_train.iterrows():
    
    if row['author'] == 'EAP':
        raw_documents_authors[0] += row['cleaned_text_string'] + ' '
    elif row['author'] == 'HPL':
        raw_documents_authors[1] += row['cleaned_text_string'] + ' '
    else:
        raw_documents_authors[2] += row['cleaned_text_string'] + ' '
        

# удалим уникальные слова, не встречающиеся у других писателей

eap_only = set(raw_documents_authors[0].split(' ')) - set(raw_documents_authors[1].split(' ')) - set(raw_documents_authors[2].split(' '))
hpl_only = set(raw_documents_authors[1].split(' ')) - set(raw_documents_authors[0].split(' ')) - set(raw_documents_authors[2].split(' '))
msh_only = set(raw_documents_authors[2].split(' ')) - set(raw_documents_authors[0].split(' ')) - set(raw_documents_authors[1].split(' '))

unique_words = eap_only.union(hpl_only).union(msh_only)

tf = TfidfVectorizer(analyzer='word')
idf_matrix =  tf.fit_transform(raw_documents_authors)
feature_names = tf.get_feature_names()
# dictionary_word = dict(zip(feature_names, idf_matrix))

dense_idf = [i.todense() for i in idf_matrix]
print(dense_idf)

max_weighted_term = []

eap_dense_list = dense_idf[0].tolist()[0]
hpl_dense_list = dense_idf[1].tolist()[0]
mws_dense_list = dense_idf[2].tolist()[0]

for inum, i in enumerate(eap_dense_list):
    max_weighted_term.append(max(hpl_dense_list[inum], mws_dense_list[inum], 
                             i))

max_tf_dict = dict(zip(feature_names, max_weighted_term))



[matrix([[ 0.00053369,  0.00053369,  0.00106738, ...,  0.00053369,
          0.        ,  0.        ]]), matrix([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.00135189,  0.00067595]]), matrix([[ 0.,  0.,  0., ...,  0.,  0.,  0.]])]


In [125]:
model = gensim.models.Word2Vec(df_train['cleaned_text'], size=100)

In [126]:
w2v = dict(zip(model.wv.index2word, model.wv.syn0))

In [127]:
w2v_array_len = list(w2v.items())[0][1].shape[0]

In [128]:
def extract_top_words(tfidfdict, numwrd):

    top_word_dict, min_value, min_key = {}, 99, ''
    

    for k, v in max_tf_dict.items():
        # print(top_word_dict.values())
        # print(v)
        if k not in unique_words and k not in eng_stopwords:
        
            if len(top_word_dict) < numwrd:
                top_word_dict[k] = v
                if v <= min_value:
                    min_key = k
            else:
                # print(v, min(list(top_word_dict.values())))
                if v > min(list(top_word_dict.values())) and k not in eng_stopwords:

                    min_value = min(top_word_dict.values())

                    for ky, va in top_word_dict.items():
                        if va == min_value:
                            min_key = ky

                    top_word_dict.pop(min_key)
                    top_word_dict[k] = v
                
    return top_word_dict
another_top_words_dict = extract_top_words(max_tf_dict, 80)
high_tf_idf_words_columns = list(another_top_words_dict.keys())


def count_topwords(target_df):

    for word in high_tf_idf_words_columns:
        
        # TODO: костыль, нужен, когда у нас уже есть такие столбцы
        # в датасете
#         try:
#             target_df = target_df.drop(word, 1)
#         except ValueError:
#             pass
        

        def count_numwords(collist):
            value = 0

            for wd in collist:
                if wd == word:
                    value += 1
            return value


        target_df[word] = target_df.cleaned_text.apply(count_numwords)
        


        

In [129]:
df_train['length']=df_train['cleaned_text_string'].apply(len)
df_train["num_words"] = df_train["text"].apply(lambda x: len(str(x).split()))
df_train["num_unique_words"] = df_train["text"].apply(lambda x: len(set(str(x).split())))
df_train["num_punctuations"] =df_train['text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))
df_train["num_words_upper"] = df_train["text"].apply(lambda x: len([w for w in str(x).split() if w.isupper()]))
df_train["num_words_title"] = df_train["text"].apply(lambda x: len([w for w in str(x).split() if w.istitle()]))
df_train["mean_word_len"] = df_train["text"].apply(lambda x: np.mean([len(w) for w in str(x).split()]))
df_train["num_stopwords"] = df_train["text"].apply(lambda x: len([w for w in str(x).lower().split() if w in eng_stopwords]))
df_train['lexical_diversity'] = df_train.text.apply(lexical_diversity)
df_train['w2v_array'] = df_train.cleaned_text.apply(avg_up_word2vec_array)
count_topwords(df_train)

df_train.head(n=3)

,id,text,author,cleaned_text,cleaned_text_string,length,num_words,num_unique_words,num_punctuations,num_words_upper,...,heart,mind,even,life,like,may,pass,first,said,friend
0,id26305,"This process, however, afforded me no means of...",EAP,"[this, process, howev, afford, mean, ascertain...",this process howev afford mean ascertain dimen...,145,41,35,7,2,...,0,0,0,0,0,0,0,0,0,0
1,id17569,It never once occurred to me that the fumbling...,HPL,"[it, never, occur, fumbl, might, mere, mistak]",it never occur fumbl might mere mistak,38,14,14,1,0,...,0,0,0,0,0,0,0,0,0,0
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP,"[in, left, hand, gold, snuff, box, caper, hill...",in left hand gold snuff box caper hill cut man...,116,36,32,5,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
def create_w2v_columns(target_df):
    
    counter = 0

    for index, row in target_df.iterrows():
        counter += 1
        
#         if counter % 100 == 0:
#             print('another hundred processed')
        
        w2v_array = row['w2v_array']
        
        for elnum, el in enumerate(w2v_array):
            target_df['w2v_feature_' + str(elnum)] = el

In [ ]:
create_w2v_columns(df_train)

In [ ]:
df_hpl=df_train[df_train['author']=='HPL']
df_hpl.describe()

In [ ]:
df_eap=df_train[df_train['author']=='EAP']
df_eap.describe()

In [ ]:
df_mws=df_train[df_train['author']=='MWS']
df_mws.describe()

In [ ]:
wordset=set()

In [ ]:
#делаю сет со всеми словами
for i in df_train.index:
    wordset |= set(df_train['cleaned_text'][i])
wordlist=list(wordset)

In [ ]:
#делаю фрейм со словами
df_word=pd.DataFrame(columns=["word", "mws", "eap", "hpl", "all"])
df_word["word"]=wordlist
df_word["mws"]=0
df_word["eap"]=0
df_word["hpl"]=0
df_word["all"]=0
df_word.head()

In [ ]:
# как мы будем эту штуку правильнее делать (возможно это жуткий костыль), я хз
# сначала создаем словарь где ключ - уникальное слово, а значение - его порядковый номер
# затем создаем разреженную матрицу, которую заполняем в зависимости от порядковых номеров 
word_dict = {}



In [ ]:
#делаю сет со всеми словами
# и сразу заготовку под шапку(потом увидишь зачем)
counter = 0
head = []

for wordlist in df_train['cleaned_text']:
    for word in wordlist:
        if word not in word_dict:
            head.append(word)
            word_dict[word] = counter
            counter += 1


In [ ]:
# видоизменять колонки в pandas руками по одному значению в строке или столбце - очень плохая идея
# колонка это numpy.ndarray, а значит при каждой итерации она будет пересоздаваться
# что угробит производительность
# делаем значит так. считаем где сколько и где встречались отдельные слова, затем создаем строку за строкой для 
# каждого предложения

list_of_lists = []

for wordlist in df_train['cleaned_text']:
    row = [0 for i in range(len(word_dict))]
    for word in wordlist:
        row[word_dict[word]] += 1
    list_of_lists.append(row)



In [ ]:
print(len(list_of_lists))

In [ ]:
# ... и для того чтобы посмотреть встречаемость того или иного слова по авторам добавим такую колонку

count_frame = pd.DataFrame(list_of_lists)
count_frame['author'] = df_train['author']



In [ ]:
# ... и теперь нормальную шапку делаем

count_frame.columns = head + ['author']


In [ ]:
print(count_frame.head())

In [ ]:
# Пока объединим все, потом может быть будем использовать
col=list(count_frame.columns)
col[-1]='author_name'
count_frame.columns=col
pivot_col=pd.pivot_table(count_frame, aggfunc=np.sum, values=col, index=['author_name'])

In [ ]:
#Убираем лишние слова, которые не учли раньше
col=list(pivot_col.columns)
col2=[string for string in col if (string[0]!='"' and string[0]!="'"
                                  and string[0]!='.' and string[0]!='`'
                                   and len(string)>3 and '.' not in string)]
col=[]
pivot_col=pivot_col[col2]
pivot_col.head()

In [ ]:
#Create pivot
pivot_col=pivot_col.append(pivot_col.sum(), ignore_index=True)
pivot_col.index=['EAP', 'HPL', 'MWS', 'SUMA']
pivot_col.head()

In [ ]:
summa=[pivot_col.loc['EAP'].sum(), pivot_col.loc['HPL'].sum(), 
       pivot_col.loc['MWS'].sum(), pivot_col.loc['SUMA'].sum()]
pivot_col['summa']=summa
pivot_col.head()

In [ ]:
# Create probability of author text knowing that a word was used
pivot_part=pivot_col
pivot_part.loc['EAP']=pivot_col.loc['EAP']/pivot_col.loc['SUMA']
pivot_part.loc['HPL']=pivot_col.loc['HPL']/pivot_col.loc['SUMA']
pivot_part.loc['MWS']=pivot_col.loc['MWS']/pivot_col.loc['SUMA']
pivot_part=pivot_part.loc[['EAP', 'HPL', 'MWS']]
# Delete unique words
pivot_part=pivot_part.loc[:, (pivot_part!=1).all(axis=0)]
pivot_part.head()

In [ ]:
# It will be easier to work this way
eap_dict=pivot_part.loc['EAP'].to_dict()
hpl_dict=pivot_part.loc['HPL'].to_dict()
mws_dict=pivot_part.loc['MWS'].to_dict()
eap_dict['word']

In [ ]:
# Create author score 
def ind_val_eap(listn):
    quant=0
    for word in listn:
        try:
            quant+=eap_dict[word]
        except KeyError:
            quant+=0
    return quant

def ind_val_hpl(listn):
    quant=0
    for word in listn:
        try:
            quant+=hpl_dict[word]
        except KeyError:
            quant+=0
    return quant

def ind_val_mws(listn):
    quant=0
    for word in listn:
        try:
            quant+=mws_dict[word]
        except KeyError:
            quant+=0
    return quant

In [ ]:
# Add index of author
df_train['mws_index']=df_train['cleaned_text'].apply(ind_val_mws)/df_train['length']
df_train['eap_index']=df_train['cleaned_text'].apply(ind_val_eap)/df_train['length']
df_train['hpl_index']=df_train['cleaned_text'].apply(ind_val_hpl)/df_train['length']
df_train.head(n=3)

In [ ]:
#Transform authors' names to numeric
df_train['author']=df_train['author'].astype('category')
df_train['author2']=df_train['author'].cat.codes
# Create different features 
df_train.head(n=3)
mid = df_train['author2']
df_train.drop(labels=['author2'], axis=1,inplace = True)
df_train.insert(0, 'author2', mid)
df_train.head()

In [ ]:
author_mapping_dict = {'EAP':0, 'HPL':1, 'MWS':2}
train_y = df_train['author'].map(author_mapping_dict)
train_id = df_train['id'].values
test_id = df_test['id'].values
cols_to_drop = ['id', 'text']
train_X = df_train.drop(cols_to_drop+['author'], axis=1)
test_X = df_test.drop(cols_to_drop, axis=1)
tfidf_vec = TfidfVectorizer(stop_words='english', ngram_range=(1,3))
full_tfidf = tfidf_vec.fit_transform(df_train['text'].values.tolist() + df_test['text'].values.tolist())
train_tfidf = tfidf_vec.transform(df_train['text'].values.tolist())
test_tfidf = tfidf_vec.transform(df_test['text'].values.tolist())

In [ ]:
def runMNB(train_X, train_y, test_X, test_y, test_X2):
    model = naive_bayes.MultinomialNB()
    model.fit(train_X, train_y)
    pred_test_y = model.predict_proba(test_X)
    pred_test_y2 = model.predict_proba(test_X2)
    return pred_test_y, pred_test_y2, model

In [ ]:
cv_scores = []
pred_full_test = 0
pred_train = np.zeros([df_train.shape[0], 3])
kf = model_selection.KFold(n_splits=3, shuffle=True, random_state=194)
for dev_index, val_index in kf.split(train_X):
    dev_X, val_X = train_tfidf[dev_index], train_tfidf[val_index]
    dev_y, val_y = train_y[dev_index], train_y[val_index]
    pred_val_y, pred_test_y, model = runMNB(dev_X, dev_y, val_X, val_y, test_tfidf)
    pred_full_test = pred_full_test + pred_test_y
    pred_train[val_index,:] = pred_val_y
    cv_scores.append(metrics.log_loss(val_y, pred_val_y))
print("Mean cv score : ", np.mean(cv_scores))
pred_full_test = pred_full_test / 5.

In [ ]:
n_comp = 50
svd_obj = TruncatedSVD(n_components=n_comp, algorithm='arpack')
svd_obj.fit(full_tfidf)
train_svd = pd.DataFrame(svd_obj.transform(train_tfidf))
test_svd = pd.DataFrame(svd_obj.transform(test_tfidf))
    
train_svd.columns = ['svd_word_'+str(i) for i in range(n_comp)]
test_svd.columns = ['svd_word_'+str(i) for i in range(n_comp)]
df_train = pd.concat([df_train, train_svd], axis=1)
df_test = pd.concat([df_test, test_svd], axis=1)
del full_tfidf, train_tfidf, test_tfidf, train_svd, test_svd

In [ ]:
### Fit transform the count vectorizer ###
tfidf_vec = CountVectorizer(stop_words='english', ngram_range=(1,3))
tfidf_vec.fit(df_train['text'].values.tolist() + df_test['text'].values.tolist())
train_tfidf = tfidf_vec.transform(df_train['text'].values.tolist())
test_tfidf = tfidf_vec.transform(df_test['text'].values.tolist())

In [ ]:
cv_scores = []
pred_full_test = 0
pred_train = np.zeros([df_train.shape[0], 3])
kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2017)
for dev_index, val_index in kf.split(train_X):
    dev_X, val_X = train_tfidf[dev_index], train_tfidf[val_index]
    dev_y, val_y = train_y[dev_index], train_y[val_index]
    pred_val_y, pred_test_y, model = runMNB(dev_X, dev_y, val_X, val_y, test_tfidf)
    pred_full_test = pred_full_test + pred_test_y
    pred_train[val_index,:] = pred_val_y
    cv_scores.append(metrics.log_loss(val_y, pred_val_y))
print("Mean cv score : ", np.mean(cv_scores))
pred_full_test = pred_full_test / 5.

# add the predictions as new features #
df_train["nb_cvec_eap"] = pred_train[:,0]
df_train["nb_cvec_hpl"] = pred_train[:,1]
df_train["nb_cvec_mws"] = pred_train[:,2]
df_test["nb_cvec_eap"] = pred_full_test[:,0]
df_test["nb_cvec_hpl"] = pred_full_test[:,1]
df_test["nb_cvec_mws"] = pred_full_test[:,2]

In [ ]:
df_train.head()

In [ ]:
del df_train['w2v_array']
df_train.columns.values

In [ ]:
# Create data set
ds_train=df_train.values
X=ds_train[:, 6:]
Y=ds_train[:, 0]
seed=7
test_size=0.3
X_train, X_test, y_train, y_test = sk.model_selection.train_test_split(X,Y, test_size=test_size, random_state=seed)

In [ ]:
# fit model
xg_train=xgb.DMatrix(X_train, label=y_train)
xg_test=xgb.DMatrix(X_test, label=y_test)
xg_t=xgb.DMatrix(X, label=Y)
param={}
param['objective'] = 'multi:softmax'
param['eta'] = 0.2
param['max_depth'] = 2
param['silent'] = 1
param['num_class'] = 3
param['eval_metric']= "mlogloss"
watchlist = [(xg_train, 'train'), (xg_test, 'test')]
num_round = 240
bst = xgb.train(param, xg_train, num_round, watchlist)
# get prediction
pred = bst.predict(xg_test)
error_rate = np.sum(pred != y_test) / y_test.shape[0]
print('Test error using softmax = {}'.format(error_rate))

In [ ]:
# do the same thing again, but output probabilities
param['objective'] = 'multi:softprob'
bstp = xgb.train(param, xg_train, num_round, watchlist)
# Note: this convention has been changed since xgboost-unity
# get prediction, this is in 1D array, need reshape to (ndata, nclass)
pred_prob = bstp.predict(xg_test).reshape(y_test.shape[0], 3)
pred_label = np.argmax(pred_prob, axis=1)
error_rate = np.sum(pred_label != y_test) / y_test.shape[0]
print('Test error using softprob = {}'.format(error_rate))

In [ ]:
# do the same thing again, but output probabilities
param['objective'] = 'multi:softprob'
bstp = xgb.train(param, xg_t, num_round, watchlist)
# Note: this convention has been changed since xgboost-unity
# get prediction, this is in 1D array, need reshape to (ndata, nclass)
pred_prob = bstp.predict(xg_t).reshape(Y.shape[0], 3)
pred_label = np.argmax(pred_prob, axis=1)
error_rate = np.sum(pred_label != Y) / Y.shape[0]
print('Test error using softprob = {}'.format(error_rate))

In [ ]:
df_test.head(n=3)

In [ ]:
df_test['cleaned_text'] = df_test.text.apply(tokenize_stem)
df_test['cleaned_text_string'] = df_test.cleaned_text.apply(' '.join)
df_test.head(n=3)

In [ ]:
df_test['length']=df_test['cleaned_text_string'].apply(len)
df_test["num_words"] = df_test["text"].apply(lambda x: len(str(x).split()))
df_test["num_unique_words"] = df_test["text"].apply(lambda x: len(set(str(x).split())))
df_test["num_punctuations"] =df_test['text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))
df_test["num_words_upper"] = df_test["text"].apply(lambda x: len([w for w in str(x).split() if w.isupper()]))
df_test["num_words_title"] = df_test["text"].apply(lambda x: len([w for w in str(x).split() if w.istitle()]))
df_test["mean_word_len"] = df_test["text"].apply(lambda x: np.mean([len(w) for w in str(x).split()]))
df_test["num_stopwords"] = df_test["text"].apply(lambda x: len([w for w in str(x).lower().split() if w in eng_stopwords]))
df_test['lexical_diversity'] = df_test.text.apply(lexical_diversity)
df_test['w2v_array'] = df_test.cleaned_text.apply(avg_up_word2vec_array)
count_topwords(df_test)
create_w2v_columns(df_test)
df_test.head(n=3)

In [ ]:
df_test['mws_index']=df_test['cleaned_text'].apply(ind_val_mws)/df_test['length']
df_test['eap_index']=df_test['cleaned_text'].apply(ind_val_eap)/df_test['length']
df_test['hpl_index']=df_test['cleaned_text'].apply(ind_val_hpl)/df_test['length']
df_test.head(n=3)
df_test.head(n=3)

In [ ]:
del df_test['cleaned_text']
del df_test['cleaned_text_string']
del df_test['w2v_array']

In [ ]:
cols=(df_train.columns.tolist())[6:]
[item for item in df_train.columns.tolist() if item not in df_test.columns.tolist()]

In [ ]:
df_test.head()

In [ ]:
df_test.columns

In [ ]:
ds_test=df_test[cols].values

In [ ]:
x_t=ds_test[:, :]
y_t=df_test['id'].values
xg_t=xgb.DMatrix(x_t)
pred_prob = bstp.predict(xg_t).reshape(y_t.shape[0], 3)
pred_label = np.argmax(pred_prob, axis=1)

In [ ]:
print(pred_prob)

In [ ]:
export=pd.DataFrame(pred_prob)
export.insert(loc=0, column='id', value=y_t)
export.columns=['id','EAP', 'HPL', 'MWS']
export.head()

In [ ]:
len(export)

In [ ]:
export[export['id']=='id23301']

In [ ]:
export.to_csv(path_or_buf="../data/export.csv", index=False)